In [1]:
import datetime

def get_current_datetime():
    # 現在の日時を取得
    dt_now = datetime.datetime.now()

    # 日時を指定された形式の文字列に変換
    str_dt_now = dt_now.strftime("%Y%m%d-%H%M")

    return str_dt_now

# この関数を呼び出して現在の日時を取得する例
# current_datetime = get_current_datetime()
# print(current_datetime)


In [2]:
#environment_setup.py
from pettingzoo.mpe import simple_speaker_listener_v4

def initialize_environment():
    # シンプルなスピーカーリスナー環境を並列環境として定義
    env = simple_speaker_listener_v4.parallel_env(continuous_actions=True)
    env.reset()
    return env


In [3]:
#environment_setup.py
import torch

def define_network_config():
    return {
        "arch": "mlp",  # ネットワークアーキテクチャ
        "h_size": [32, 32],  # アクターの隠れ層のサイズ
    }

def define_initial_hyperparameters():
    return {
        "POPULATION_SIZE": 4,
        "ALGO": "MATD3",  # Algorithm
        # Swap image channels dimension from last to first [H, W, C] -> [C, H, W]
        "CHANNELS_LAST": False,
        "BATCH_SIZE": 32,  # Batch size
        "LR": 0.01,  # Learning rate
        "GAMMA": 0.95,  # Discount factor
        "MEMORY_SIZE": 100000,  # Max memory buffer size
        "LEARN_STEP": 5,  # Learning frequency
        "TAU": 0.01,  # For soft update of target parameters
        "POLICY_FREQ": 2,  # Policy frequnecy
        # ... (INIT_HP辞書の残り)
    }

def get_device():
    #return torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return torch.device("mps")


In [4]:
#agents.py
from agilerl.utils.utils import initialPopulation

def set_action_and_state_dimensions(env, INIT_HP):
    """
    環境から行動次元と状態次元を設定し、初期ハイパーパラメータを更新する。
    env: 学習環境
    init_hp: 初期ハイパーパラメータの辞書
    """
    try:
        # まず、状態次元を設定する
        # 状態空間が離散的か連続的かに基づいて状態次元を取得する
        state_dim = [env.observation_space(agent).n for agent in env.agents]
        one_hot = True
    except Exception:
        # 連続的な状態空間の場合
        state_dim = [env.observation_space(agent).shape for agent in env.agents]
        one_hot = False

    try:
        # 次に、行動次元を設定する
        # 行動空間が離散的か連続的かに基づいて行動次元を取得する
        action_dim = [env.action_space(agent).n for agent in env.agents]
        INIT_HP["DISCRETE_ACTIONS"] = True
        INIT_HP["MAX_ACTION"] = None
        INIT_HP["MIN_ACTION"] = None
    except Exception:
        # 連続的な行動空間の場合
        action_dim = [env.action_space(agent).shape[0] for agent in env.agents]
        INIT_HP["DISCRETE_ACTIONS"] = False
        INIT_HP["MAX_ACTION"] = [env.action_space(agent).high for agent in env.agents]
        INIT_HP["MIN_ACTION"] = [env.action_space(agent).low for agent in env.agents]

    if False: # デバッグ出力
        print("state_dim", state_dim)
        print("one_hot", one_hot)
        print( 'INIT_HP["DISCRETE_ACTIONS"]', INIT_HP["DISCRETE_ACTIONS"])
        print( 'INIT_HP["MAX_ACTION"]', INIT_HP["MAX_ACTION"])
        print( 'INIT_HP["MIN_ACTION"]', INIT_HP["MIN_ACTION"])

    # 状態次元の調整（CHANNELS_LAST オプションが True の場合）
    if INIT_HP["CHANNELS_LAST"]:
        state_dim = [
            (state_dim[2], state_dim[0], state_dim[1]) for state_dim in state_dim
        ]

    return state_dim, action_dim, INIT_HP, one_hot

# Create a population ready for evolutionary hyper-parameter optimisation
# 進化的なハイパーパラメータ最適化のための母集団を作成する
def create_population(INIT_HP, state_dim, action_dim, one_hot, NET_CONFIG, device):
    return initialPopulation(
        INIT_HP["ALGO"],
        state_dim,
        action_dim,
        one_hot,
        NET_CONFIG,
        INIT_HP,
        population_size=INIT_HP["POPULATION_SIZE"],
        device=device,
        # ... (その他のパラメータ)
    )


In [5]:
#replay_buffer.py
from agilerl.components.multi_agent_replay_buffer import MultiAgentReplayBuffer

def setup_replay_buffer(INIT_HP, device):
    field_names = ["state", "action", "reward", "next_state", "done"]
    return MultiAgentReplayBuffer(
        INIT_HP["MEMORY_SIZE"],
        field_names=field_names,
        agent_ids=INIT_HP["AGENT_IDS"],
        device=device,
        # ... (その他のパラメータ)
    )


In [6]:
#hpo.py
from agilerl.hpo.mutation import Mutations
from agilerl.hpo.tournament import TournamentSelection

# Instantiate a tournament selection object (used for HPO)
def tournament_selection(INIT_HP):
    return TournamentSelection(
        tournament_size=2,  # Tournament selection size
        elitism=True,  # Elitism in tournament selection
        population_size=INIT_HP["POPULATION_SIZE"],  # Population size
        evo_step=1,
        # ... (その他のパラメータ)
    )# Evaluate using last N fitness scores

# Instantiate a mutations object (used for HPO)
def mutations_config(INIT_HP, NET_CONFIG, device):
    return Mutations(
        algo=INIT_HP["ALGO"],
        no_mutation=0.2,  # Probability of no mutation
        architecture=0.2,  # Probability of architecture mutation
        new_layer_prob=0.2,  # Probability of new layer mutation
        parameters=0.2,  # Probability of parameter mutation
        activation=0,  # Probability of activation function mutation
        rl_hp=0.2,  # Probability of RL hyperparameter mutation
        rl_hp_selection=[
            "lr",
            "learn_step",
            "batch_size",
        ],  # RL hyperparams selected for mutation
        mutation_sd=0.1,  # Mutation strength
        agent_ids=INIT_HP["AGENT_IDS"],
        arch=NET_CONFIG["arch"],
        rand_seed=2,#1,
        device=device,
        # ... (その他のパラメータ)
    )


In [9]:
#train.py
import numpy as np
from tqdm import trange

def training_loop(env, pop ,memory, tournament, mutations, INIT_HP, net_config, max_episodes, max_steps):
    
    # Define training loop parameters
    # 学習ループ・パラメータを定義
    max_episodes = 10 # 6000 #500  # Total episodes (default: 6000)
    max_steps = 100 #25  # Maximum steps to take in each episode
    epsilon = 1.0  # Starting epsilon value
    eps_end = 0.1  # Final epsilon value
    eps_decay = 0.995  # Epsilon decay
    evo_epochs = 20  # Evolution frequency
    evo_loop = 1  # Number of evaluation episodes
    elite = population[0]  # Assign a placeholder "elite" agent
    
    # Training loop
    # 学習ループ
    for idx_epi in trange(max_episodes):
        
        for agent in population:  # Loop through population
            
            state, info = env.reset()  # Reset environment at start of episode
            agent_reward = {agent_id: 0 for agent_id in env.agents}
            if INIT_HP["CHANNELS_LAST"]:
                state = {
                    agent_id: np.moveaxis(np.expand_dims(s, 0), [-1], [-3])
                    for agent_id, s in state.items()
                }

            for _ in range(max_steps):
                agent_mask = info["agent_mask"] if "agent_mask" in info.keys() else None
                env_defined_actions = (
                    info["env_defined_actions"]
                    if "env_defined_actions" in info.keys()
                    else None
                )

                # Get next action from agent
                # エージェントから次の行動を取得する
                cont_actions, discrete_action = agent.getAction(
                    state, epsilon, agent_mask, env_defined_actions
                )
                if agent.discrete_actions:
                    action = discrete_action
                else:
                    action = cont_actions

                next_state, reward, termination, truncation, info = env.step(
                    action
                )  # Act in environment

                # Image processing if necessary for the environment
                # 環境に応じた画像処理を行う
                if INIT_HP["CHANNELS_LAST"]:
                    state = {agent_id: np.squeeze(s) for agent_id, s in state.items()}
                    next_state = {
                        agent_id: np.moveaxis(ns, [-1], [-3])
                        for agent_id, ns in next_state.items()
                    }

                # Save experiences to replay buffer
                # 経験をリプレイバッファに保存する
                memory.save2memory(state, cont_actions, reward, next_state, termination)

                # Collect the reward
                # 報酬を受け取る
                for agent_id, r in reward.items():
                    agent_reward[agent_id] += r

                # Learn according to learning frequency
                # 学習周期に合わせて学習する
                if (memory.counter % agent.learn_step == 0) and (
                    len(memory) >= agent.batch_size
                ):
                    experiences = memory.sample(
                        agent.batch_size
                    )  # Sample replay buffer
                    agent.learn(experiences)  # Learn according to agent's RL algorithm

                # Update the state
                # 状態を更新する
                if INIT_HP["CHANNELS_LAST"]:
                    next_state = {
                        agent_id: np.expand_dims(ns, 0)
                        for agent_id, ns in next_state.items()
                    }
                state = next_state

                # Stop episode if any agents have terminated
                # いずれかのエージェントが終了したならば、エピソードを停止する
                if any(truncation.values()) or any(termination.values()):
                    break

            # Save the total episode reward
            # エピソードの合計報酬を保存する
            score = sum(agent_reward.values())
            agent.scores.append(score)

        # Update epsilon for exploration
        # 探索用のイプシロンを更新
        epsilon = max(eps_end, epsilon * eps_decay)

        # Now evolve population if necessary
        # 必要であれば、母集団を進化させる
        if (idx_epi + 1) % evo_epochs == 0:
            # Evaluate population
            fitnesses = [
                agent.test(
                    env,
                    swap_channels=INIT_HP["CHANNELS_LAST"],
                    max_steps=max_steps,
                    loop=evo_loop,
                )
                for agent in population
            ]

            print(f"Episode {idx_epi + 1}/{max_episodes}")
            print(f'Fitnesses: {["%.2f" % fitness for fitness in fitnesses]}')
            print(
                f'100 fitness avgs: {["%.2f" % np.mean(agent.fitness[-100:]) for agent in population]}'
            )

            # Tournament selection and population mutation
            # トーナメント選択と母集団の変異
            elite, population = tournament.select(population)
            population = mutations.mutation(population)

    # Save the trained algorithm
    # 学習アルゴリズムを保存する
    #path = "./models/MATD3"
    path = "./"+str_dt_now+"/models/MATD3"
    filename = "MATD3_trained_agent.pt"
    os.makedirs(path, exist_ok=True)
    save_path = os.path.join(path, filename)
    elite.saveCheckpoint(save_path)

In [10]:
#main.py
import datetime
import numpy as np
from tqdm import trange
import os

import pprint

# モジュールをインポート
#import environment_setup
#import config
#import agents
#import replay_buffer
#import hpo
#import train

def main():
    device = get_device()
    
    print("===== AgileRL Online Multi-Agent Demo =====")
    
    NET_CONFIG = define_network_config()
    INIT_HP = define_initial_hyperparameters()
    env = initialize_environment()
    
    # Append number of agents and agent IDs to the initial hyperparameter dictionary
    # エージェントとエージェントIDの数を、ハイパーパラメータディクショナリの初期化のために加える
    INIT_HP["N_AGENTS"] = env.num_agents
    INIT_HP["AGENT_IDS"] = env.agents
    
    if True: # デバッグ出力
        print('state_dim', state_dim)
        print('action_dim', action_dim)
        print('one_hot', one_hot)
        print('NET_CONFIG')
        pprint.pprint(NET_CONFIG)
        print('INIT_HP')
        pprint.pprint(INIT_HP)
        print('device', device)
    
    state_dim, action_dim, INIT_HP, one_hot = set_action_and_state_dimensions(env, INIT_HP)
    pop = create_population(INIT_HP["ALGO"], state_dim, action_dim, one_hot, NET_CONFIG, INIT_HP, INIT_HP["POPULATION_SIZE"], device)
    memory = setup_replay_buffer(INIT_HP, env.agent, device=device)
    tournament = tournament_selection(INIT_HP)
    mutations = mutations_config(INIT_HP, NET_CONFIG)
    
    training_loop(env, pop ,memory, tournament, mutations, INIT_HP, NET_CONFIG, max_episodes=6000, max_steps=25)
    
    
    # ... (トレーニングループの残りのコード)

if __name__ == "__main__":
    main()


===== AgileRL Online Multi-Agent Demo =====


NameError: name 'define_network_config' is not defined

In [ ]:
#environment.py
from pettingzoo.mpe import simple_speaker_listener_v4

def setup_environment(render_mode="rgb_array"):
    env = simple_speaker_listener_v4.parallel_env(continuous_actions=True, render_mode=render_mode)
    env.reset()
    return env


In [ ]:
#utils.py
from PIL import Image, ImageDraw

def label_with_episode_number(frame, episode_num):
    im = Image.fromarray(frame)
    drawer = ImageDraw.Draw(im)
    text_color = (255, 255, 255) if np.mean(frame) < 128 else (0, 0, 0)
    drawer.text((im.size[0] / 20, im.size[1] / 18), f"Episode: {episode_num+1}", fill=text_color)
    return im


In [ ]:
#agent.py
from agilerl.algorithms.matd3 import MATD3

def load_matd3_agent(path, state_dim, action_dim, one_hot, n_agents, agent_ids, max_action, min_action, discrete_actions, device):
    matd3 = MATD3(state_dim, action_dim, one_hot, n_agents, agent_ids, max_action, min_action, discrete_actions, device=device)
    matd3.loadCheckpoint(path)
    return matd3


In [ ]:
#main.py
import os
import imageio
import numpy as np
import torch

# モジュールのインポート
import environment
import utils
import agent

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    env = environment.setup_environment()
    # ... (残りのメインスクリプトコード)

if __name__ == "__main__":
    main()
